In [1]:
from logikon.backends.chat_models_with_grammar import create_logits_model

In [2]:
import dotenv
import os

dotenv.load_dotenv("../.env")
"HUGGINGFACEHUB_API_TOKEN" in os.environ or print("Please set HUGGINGFACEHUB_API_TOKEN in .env")

True

In [3]:
kwargs = {
    "model_id": "HuggingFaceH4/zephyr-7b-beta",
    "inference_server_url": "https://px0zqc1h7zw38b0b.us-east-1.aws.endpoints.huggingface.cloud",
    "llm_backend": "HFChat",
    "api_key": os.environ.get('HUGGINGFACEHUB_API_TOKEN'),
    "temperature": 0.7,
}
hf_chat = create_logits_model(**kwargs)


/Users/gregorbetz/git/logikon2/.direnv/logikon/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [7]:
from langchain_core.messages import HumanMessage
res = await hf_chat.get_labelprobs(
    [HumanMessage(content="What is the capital of Canada?\n(A) Paris\n(B) Lyon\n(C) Quebec")],
    labels=["A", "B", "C"],
    top_logprobs=5
)

In [8]:
res

{'A': 0.043252036708846336, 'B': 0.11174987599215074, 'C': 0.8449980872990028}